In [ ]:
import os
import sys
os.environ['OMP_NUM_THREADS'] = '1'
os.environ["THEANO_FLAGS"] = "device=gpu"
sys.path.append(os.getcwd()+"/..")
import lasagne
import theano.tensor as T
import theano
from tasks import check
from scripts import manifold
from lasagne import layers as L

from lasagnekit.misc.plot_weights import dispims_color, tile_raster_images
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.io import imsave

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.manifold import TSNE
from image_scatter import image_scatter
from tqdm import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import tile_raster_images
import numpy as np

from helpers import salt_and_pepper
def sample(x):
    return (np.random.uniform(size=x.shape) <= x).astype(np.float32)


def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, max(len(inputs) - batchsize + 1, len(inputs)), batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

## load model

In [ ]:
v = check(
    what="notebook", 
    filename="../training/fonts/dense2/model.pkl",
    force_w=28,
    force_h=28,
    dataset="fonts"
)

In [ ]:
capsule, data, layers, w, h, c = v

In [ ]:
X = T.tensor4()
encode = theano.function([X], L.get_output(layers['hid2'], X))

## viz features

In [ ]:
for name, layer in layers.items():
    if hasattr(layer, "W"):
        W = layer.W.get_value()
        if len(W.shape)==2:
            nbpixels = c*w*h
            if W.shape[0] == nbpixels:
                W = W.T
            a, b = w, h
        else:
            W = W.reshape((W.shape[0]*W.shape[1], W.shape[2], W.shape[3]))
            a, b = W.shape[1:]
        fig = plt.figure(figsize=(10, 10))
        s = int(np.sqrt(W.shape[0]))
        plt.axis('off')
        img=tile_raster_images(W, (a, b), (s, s))
        plt.imshow(img, cmap="gray", interpolation='none')
        plt.show()

## iterative refinement

In [ ]:
seed = 42
np.random.seed(seed)
N = 100
s = np.random.uniform(0, 1, size=(N, c, w, h))
s = s.astype(np.float32)
samples = []
samples.append(s.copy()[None, :, :, :, :])
nb_iter = 60
loss = []
for i in tqdm(range(nb_iter)):
    sprev = s  
    #s = s > 0.6
    s = capsule.reconstruct(s)
    s = s.astype(np.float32)
    samples.append(s.copy()[None, :, :, :, :])
    s = np.random.binomial(n=1, p=s, size=s.shape).astype('float32')# binarize by sampling
    #s = s > 0.6
    score = np.abs(s - sprev).sum()
    loss.append(score)
samples = np.concatenate(samples, axis=0)
print(samples.shape)
print(loss)

In [ ]:
fig = plt.figure(figsize=(20, 20))
h = 28
sw, sh = samples.shape[0:2]
samples_ = samples.copy()
samples_ = samples_.reshape((sw*sh, c, w, h))
samples_ = samples_.transpose((0, 2, 3, 1))
samples_ = samples_ * np.ones((1, 1, 1, 3))
img = dispims_color(samples_)
imsave('out.png', img)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
h = []
d = []
for i in range(10):
    d.append(capsule.preprocess(data.X).transpose((0, 2 ,3 , 1))  * np.ones((1, 1, 1, 3)))
    h.append(encode(capsule.preprocess(data.X)))

d = np.concatenate(d, axis=0)
h = np.concatenate(h, axis=0)

In [ ]:
#mf = IncrementalPCA(n_components=2, batch_size=1024)
mf = PCA(n_components=2)
#mf = TSNE(n_components=2, verbose=1)
h2d = mf.fit_transform(h)

In [ ]:
mf = PCA(n_components=2)
#mf = IncrementalPCA(n_components=2, batch_size=1024)
#mf = TSNE(n_components=2, verbose=1)
d2d = mf.fit_transform(d.reshape(d.shape[0], -1))

In [ ]:
img = image_scatter(h2d, d, 80)
imsave('out_scatter.png', img)
plt.imshow(img, cmap='gray')
plt.show()

In [ ]:
img = image_scatter(d2d, d, 80)
imsave('out_scatter_data.png', img)
plt.imshow(img, cmap='gray')
plt.show()

## transferability

In [ ]:
from lasagnekit.datasets.mnist import MNIST

In [ ]:
mnist = MNIST()
mnist.load()
X_transf_orig = mnist.X.reshape((mnist.X.shape[0], c, w, h))
X_transf_orig = X_transf_orig[mnist.y==5]
X_transf_orig = X_transf_orig[0:500]

In [ ]:
X_transf_rec = X_transf_orig.copy().astype(np.float32)
X_transf_rec = X_transf_rec
for i in range(1):
    X_transf_rec = capsule.reconstruct(X_transf_rec)
    print(((X_transf_rec - capsule.reconstruct(X_transf_rec))**2).sum(axis=(1, 2, 3)).mean())

In [ ]:
X_transf = X_transf_orig.transpose((0, 2, 3, 1))
X_transf = X_transf * np.ones((1, 1, 1, 3))

X_transf_rec = X_transf_rec.transpose((0, 2, 3, 1))
X_transf_rec = X_transf_rec * np.ones((1, 1, 1, 3))

In [ ]:
fig = plt.figure(figsize=(20, 20))
img = dispims_color(X_transf_rec[0:500])
plt.imshow(img, cmap='gray')
plt.show()
fig = plt.figure(figsize=(20, 20))
img = dispims_color(X_transf[0:500])
plt.imshow(img, cmap='gray')
plt.show()

## moment matching

In [ ]:
from lasagne.nonlinearities import rectify, sigmoid
from lasagne import updates
z_dim = 10
Z = T.matrix()
h=28
z_in = L.InputLayer((None, z_dim))
h1 = L.DenseLayer(z_in, num_units=100, nonlinearity=rectify)
h2 = L.DenseLayer(h1, num_units=500, nonlinearity=rectify)
x = L.DenseLayer(h2, num_units=c*w*h, nonlinearity=sigmoid)
x = L.ReshapeLayer(x, ([0], c, w, h))
gen = theano.function([Z], L.get_output(x, Z))


def transform_r(s):
    return s.mean(axis=0)
    rr = s[:, None, :]
    rrr = s[:, :, None]
    return (rr*rr).mean(axis=0)

rx = L.get_output(layers['hid2'], L.get_output(x, Z))
rX = L.get_output(layers['hid2'], X)


loss = ((rx - rX) ** 2).mean()
#loss = ((transform_r(rx) - transform_r(rX))**2).sum()
#loss = ((rx.mean(axis=0) - rX.mean(axis=0))**2).sum()
#loss = (((rx**2).mean(axis=0) - (rX**2).mean(axis=0))**2).sum()



#loss = ((rx - rX)**2).mean()

params = L.get_all_params(x, trainable=True)
grad_updates = updates.adam(loss, params, learning_rate=0.01)
train_fn = theano.function([Z, X], loss, updates=grad_updates)
loss_fn = theano.function([Z, X], loss)

In [ ]:
from lasagnekit.datasets.helpers import split
from time import time
print('Loading data...')

def preprocess(data):
    return data.reshape((data.shape[0], c, w, h))

train_full = MNIST(which='train')
train_full.load()
train_full.X = preprocess(train_full.X)
#train_full.X = train_full.X[0:128*2]

train, valid = split(train_full, test_size=0.16667) # 10000 examples in validation set

test = MNIST(which='test')
test.load()
test.X = preprocess(test.X)

In [ ]:
from collections import OrderedDict
from tabulate import tabulate
from
nb_epochs = 100
batchsize = 128
#learning_rate.set_value(np.array(lr).astype(np.float32))
history = []
print('Training...')


z_full = 
z_full = np.random.uniform(-1, 1, size=(train_X_full.shape[0], z_dim)).astype(np.float32)

for epoch in range(1, nb_epochs + 1):
    train_time = []
    train_X_full = train.X

    train_y_full = train.y
    for train_X, z in iterate_minibatches(train_X_full, z_full, batchsize):
        # Train one mini=batch
        t = time()
        train_fn(z, train_X)
        train_time.append(time() - t)
    
    stats = OrderedDict()
    z = np.random.uniform(-1, 1, size=(train.X.shape[0], z_dim)).astype(np.float32)
    stats['train_loss'] = loss_fn(z, train.X)
    z = np.random.uniform(-1, 1, size=(valid.X.shape[0], z_dim)).astype(np.float32)
    stats['valid_losss'] = loss_fn(z, valid.X)
    stats['train_time'] = np.sum(train_time)
    stats['epoch'] = epoch
    
    history.append(stats)
    print(tabulate([stats], headers="keys"))
    #lr = lr * lr_decay
    #learning_rate.set_value(np.array(lr).astype(np.float32))

In [ ]:
z = np.random.uniform(-1, 1, size=(256, z_dim)).astype(np.float32)
xgen = gen(z)
xgen = xgen.reshape((xgen.shape[0], c, w, h))
xgen = xgen.transpose((0, 2, 3, 1))
xgen = xgen * np.ones((1, 1, 1, 3))
fig = plt.figure(figsize=(20, 20))
img = dispims_color(xgen, border=1)
plt.imshow(img)